In [1]:
!pip install torch

In [2]:
from shutil import copyfile
copyfile('drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/data/archive.hdf5','archive.hdf5')

'archive.hdf5'

In [3]:
import os
import re

import h5py
import numpy as np
import torch.utils.data
from PIL import Image

import logging
import os
logger = logging.getLogger()
logger.setLevel(os.environ.get("LOGLEVEL", "INFO"))


class HiroshimaLemon(torch.utils.data.Dataset):
    """HiroshimaLemon dataset."""

    def __init__(self, data='archive.hdf5', split='train', in_memory=True):
        assert os.path.exists(data), "Data path '{}' not found".format(data)
        splits = ["train", "test"]
        assert split in splits, "Split '{}' not supported".format(split)
        logger.info("Constructing HiroshimaLemon {}...".format(split))
        self._data, self._split = data, split
        self._size = 0
        assert in_memory, "Only in_memory implemented"
        self._in_memory = in_memory
        self._construct_imdb()

    def _construct_imdb(self):
        """Constructs the imdb."""

        with h5py.File(self._data,'r') as hf:

            if self._in_memory:
                self._imdb = {}
            if self._split == 'train':
                self._imdb_lookup = {}
                i = 0
                train = hf['train']
                self._class_ids = sorted(f for f in train.keys() if re.match(r"^[0-9]+$", f))
                self._class_id_cont_id = {v: i for i, v in enumerate(self._class_ids)}
                for class_id in self._class_ids:
                    cont_id = self._class_id_cont_id[class_id]
                    if self._in_memory:
                        self._imdb[cont_id] = train[class_id][()]
                    size = train[class_id].shape[0]
                    self._imdb_lookup.update(dict(zip(list(range(i,i+size)),list(zip(list(range(0,size)),[cont_id]*size)))))
                    i += size
                self._size = i
                logger.info("Number of images: {}".format(i))
                logger.info("Number of classes: {}".format(len(self._class_ids)))
            else:
                if self._in_memory:
                    self._imdb = hf['test'][()]
                    size = self._imdb.shape[0]
                    logger.info("Number of images: {}".format(size))
                    self._size = size

    def __getitem__(self, index):
        assert index >= 0 and index < self._size, \
            "Provided index {} must be in range [0, {}).".format(index, self._size)
        if self._split == 'train':
          i, cont_id = self._imdb_lookup[index]
          return {'input':Image.fromarray(self._imdb[cont_id][i,:,:,:], 'RGB'),'target':cont_id}
        else:
          return {'input':self._imdb[index,:,:,:]}

    def __len__(self):
        return self._size

In [4]:
dataset_train = HiroshimaLemon(split='train')
dataset_test = HiroshimaLemon(split='test')

x,y = np.meshgrid(np.linspace(-310,330,640),np.linspace(-330,310,640))
x = x[:,:,None]
y = y[:,:,None]

In [5]:
import cv2

def kmeans_color_quantization(image, clusters=8, rounds=10):
    h, w = image.shape[:2]
    samples = np.zeros([h*w,3], dtype=np.float32)
    count = 0

    for x in range(h):
        for y in range(w):
            samples[count] = image[x][y]
            count += 1

    compactness, labels, centers = cv2.kmeans(samples,
            clusters, 
            None,
            (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10000, 0.01), 
            rounds, 
            cv2.KMEANS_RANDOM_CENTERS)

    centers = np.uint8(centers)
    res = centers[labels.flatten()]
    return res.reshape((image.shape))

In [6]:
def filter(im,x,y):

  im = np.where(np.sqrt(x**2+y**2)<280,im,0)

  min_percent = 50   # Low percentile
  max_percent = 100  # High percentile
  lo, hi = np.percentile(im, (min_percent, max_percent))
  # Apply linear "stretch" - lo goes to 0, and hi goes to 1
  im = (im.astype(float) - lo) / (hi-lo)
  #Multiply by 255, clamp range to [0, 255] and convert to uint8
  im = np.maximum(np.minimum(im*255, 255), 0).astype(np.uint8)

  lower =(0, 0, 0) # lower bound for each channel
  upper = (35, 35, 35) # upper bound for each channel
  # create the mask and use it to change the colors
  mask = ~cv2.inRange(im, lower, upper)
  """
  kmeans = kmeans_color_quantization(im, clusters=2, rounds=10)[:,:,0]
  kmeans = np.where(kmeans<255//2,0,1).astype(np.uint8)
  """
  kernel= cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(20,20))
  mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
  mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
  #kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
  #mask = cv2.dilate(mask,kernel,iterations = 1)
  return mask.astype(bool)

In [8]:
import matplotlib.pyplot as plt
import cv2
from random import choice

for _ in range(50):
    while True:
      n = choice(range(len(dataset_train)))
      im = dataset_train[n]['input']
      target = dataset_train[n]['target']
      if target!=3:
        continue
      im = np.array(im)
      print(n)

      mask = filter(im.copy(),x,y)

      fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(16, 5))
      axs = axs.flatten()
      axs[0].imshow(im)
      axs[1].imshow(mask[:,:,None]*im)
      axs[2].imshow(mask)
      plt.show()
      break

Output hidden; open in https://colab.research.google.com to view.

In [9]:
import h5py
from tqdm import tqdm

im_list, mask_list = [], []
for name, dataset in [('test',dataset_test),('train',dataset_train)]:
  for i in tqdm(range(len(dataset))):

    """
    bad filter results, manually fix later instead of skipping!
    if name == 'train':
      if i in [896,1038,931,932,964,1022,1047]:
        continue
    """

    im = np.array(dataset[i]['input'])
    mask = filter(im,x,y)

    im_list.append(mask[:,:,None]*im)
    mask_list.append(mask)

100%|██████████| 1102/1102 [00:58<00:00, 18.76it/s]


In [10]:
# Free up memory for variables without references
del dataset_test
del dataset_train
del x
del y

In [11]:
# Write to hdf5 file in two parts, to limit memory usage
with h5py.File('archive_mask.hdf5', 'w') as hf:
  shape = (len(im_list),640,640,3)
  hf.create_dataset('im',
                  data=np.stack(im_list,axis=0),
                  shape=shape,
                  maxshape=shape,
                  compression="gzip",
                  compression_opts=9,
                  chunks=True,
                  dtype=np.uint8)
  hf.close()
  del im_list

with h5py.File('archive_mask.hdf5', 'a') as hf:
  shape = (len(mask_list),640,640)
  hf.create_dataset('mask',
                  data=np.stack(mask_list,axis=0),
                  shape=shape,
                  maxshape=shape,
                  compression="gzip",
                  compression_opts=9,
                  chunks=True,
                  dtype=np.bool)
  hf.close()
  del mask_list

In [12]:
copyfile('archive_mask.hdf5','drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/data/archive_mask.hdf5')

'drive/MyDrive/Colab Notebooks/SIGNATEHiroshimaLemon/data/archive_mask.hdf5'